In [1]:
import numpy as np
from scipy.special import factorial, factorial2
from sympy.physics.wigner import wigner_9j, wigner_6j, wigner_3j
from itertools import product

from moshinsky_way import set_moshinsky_brackets

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


## Test Moshinsky brackets

In [2]:
# Test that our Moshinsky brackets are correct (quite incomplete but oh well)

loaded_brackets = set_moshinsky_brackets(10, 10)

si = np.sqrt(1/2)
co = -si

def A(n, l):
    return (-1)**n * (factorial2(2*n) * factorial2(2*n + 2*l + 1))**(-1/2)


In [3]:

# l1 = 0
# l2 = 0
# n1 = 0
# n2 = 0
# lamb = 0

count = 0
for l1, l2 in product(range(0, 4), repeat=2):
    for n1, n2 in product(range(0, 4), repeat=2):
        for lamb in range(np.abs(l1-l2), l1+l2+1):
        
            Nq = 2*n1 + l1 + 2*n2 + l2

            # if (l1 + l2) % 2 != 1:
            #     continue

            if (Nq - lamb) % 2 != 0:
                continue

            n_prime = (Nq - lamb)//2

            if 2*n_prime + lamb != Nq:
                print("WTF")

            bracket = float(co**(2*n1 + l1) * si**(2*n2 + l2) * (-1)**lamb * ((2*l1 + 1) * (2*l2 + 1))**(1/2) *\
                        wigner_3j(l1, l2, lamb, 0, 0, 0).evalf() * A(n1, l1) * A(n2, l2) / A(n_prime, lamb))


            loaded_bracket = loaded_brackets[n_prime, lamb, 0, 0, n1, l1, l2, lamb]
            loaded_bracket_2 = loaded_brackets[n1, l1, n2, l2, n_prime, lamb, 0, lamb]

            #print(bracket, loaded_bracket)

            if not np.isclose(loaded_bracket, loaded_bracket_2, atol=1e-3):
                print("WTF")
                print(loaded_bracket, loaded_bracket_2)

            if not np.isclose(bracket, loaded_bracket, atol=1e-3):
                print("Error in Moshinsky brackets:", n1, l1, n2, l2, n_prime, lamb)
                print(bracket, loaded_bracket)
                #print(A(n1, l1), A(n2, l2), A(n_prime, lamb), wigner_3j(l1, l2, lamb, 0, 0, 0).evalf())
                print("even" if (l1 + l2) % 2 == 0 else "odd")
                print(Nq)
            
            count += 1

print("COUNT:", count)

COUNT: 480


## Test the index flattener/unflattener

In [2]:
import full_minnesota_hf_system as hfs

system = hfs.System(Ne_max=4, l_max=1, hbar_omega=10, mass=939)

print("Num_sates:", system.num_states)
indices = np.arange(0, system.num_states)

unflattened = []
for idx in indices:
    unflatten = system.index_unflattener(idx)
    flatten = system.index_flattener(*unflatten)
    if idx != flatten:
        print("WTF")
    
    unflattened.append(unflatten)
# All unique?
print(len(unflattened), len(set(unflattened)))


idx_list = []
for n in range(system.n_level_max + 1):
    l_max = min(system.l_level_max, system.Ne_max - 2*n)
    for l in range(l_max + 1):
        for twoj in range(np.abs(2 * l - 1), 2 * l + 2, 2):
            for twom in range(-twoj, twoj + 1, 2):
                
                flatten = system.index_flattener(n, l, twoj, twom)
                unflatten = system.index_unflattener(flatten)

                if (n, l, twoj, twom) != unflatten:
                    print("WTF 2")
                
                #print(flatten, unflatten)
                idx_list.append(flatten)

idx_list = np.array(idx_list)
print(idx_list)
print(indices)

# print(system.index_unflattener(18))
# print(system.index_unflattener(19))
print(system.index_flattener(0, 1, 3, 3))


Num_sates: 18
18 18
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
7


In [104]:
#system = hfs.System(Ne_max=2, l_max=1, hbar_omega=10, mass=939)
print("Num_sates:", system.num_states)
ns = system.num_states

np.random.seed(0)
mat = np.random.rand(system.n_level_max+1, system.l_level_max+1, 2, system.n_level_max+1, system.l_level_max+1, 2)

m_diagonal = True
mat_flat = system.matrix_ndflatten(mat, dim=2, m_diagonal=m_diagonal, asym=False)

print(mat_flat.shape)

consistent = True
for unf1 in unflattened:
    for unf2 in unflattened:
        idx1 = system.index_flattener(*unf1)
        idx2 = system.index_flattener(*unf2)

        n1, l1 = unf1[0], unf1[1]
        n2, l2 = unf2[0], unf2[1]

        twoj1_idx = 0 if unf1[2] < 2 * l1 else 1
        twoj2_idx = 0 if unf2[2] < 2 * l2 else 1

        # print(idx1, idx2)
        # print(unf1, unf2)
        # print(mat[n1, l1, twoj1_idx, n2, l2, twoj2_idx])
        # print(mat_flat[idx1, idx2])

        if m_diagonal:
            m1, m2 = unf1[3], unf2[3]
            if m1 != m2:
                if np.isclose(mat_flat[idx1, idx2], 0):
                    continue
                    
                print("m diagonal not working")
                consistent = False

        if not np.isclose(mat[n1, l1, twoj1_idx, n2, l2, twoj2_idx], mat_flat[idx1, idx2]):
            print("WTF")
            consistent = False

print("2D flattening is consistent:", consistent)


Num_sates: 18
(18, 18)
2D flattening is consistent: True


In [6]:
import full_minnesota_numba as hfn
# 4D flattening
np.random.seed(0)
mat = np.random.rand(system.n_level_max+1, system.l_level_max+1, 2, system.n_level_max+1, system.l_level_max+1, 2,
                        system.n_level_max+1, system.l_level_max+1, 2, system.n_level_max+1, system.l_level_max+1, 2)

m_diagonal = True
asym = True
mat_flat = system.matrix_ndflatten(mat, dim=4, m_diagonal=m_diagonal, asym=asym)
mat_flat_2 = hfn.n_matrix_4dflatten(mat, m_diagonal, asym, system.num_states, system.Ne_max, system.l_level_max)
print("Both flattening systems equivalent:", np.allclose(mat_flat, mat_flat_2))

print(mat_flat.shape)

consistent = True
for unf1, unf2, unf3, unf4 in product(unflattened, repeat=4):
        idx1 = system.index_flattener(*unf1)
        idx2 = system.index_flattener(*unf2)
        idx3 = system.index_flattener(*unf3)
        idx4 = system.index_flattener(*unf4)

        n1, l1 = unf1[0], unf1[1]
        n2, l2 = unf2[0], unf2[1]
        n3, l3 = unf3[0], unf3[1]
        n4, l4 = unf4[0], unf4[1]

        twoj1_idx = 0 if unf1[2] < 2 * l1 else 1
        twoj2_idx = 0 if unf2[2] < 2 * l2 else 1
        twoj3_idx = 0 if unf3[2] < 2 * l3 else 1
        twoj4_idx = 0 if unf4[2] < 2 * l4 else 1

        if m_diagonal:
            m1, m2, m3, m4 = unf1[3], unf2[3], unf3[3], unf4[3]
            if not asym:
                if m1 != m3 or m2 != m4:
                    if np.isclose(mat_flat[idx1, idx2, idx3, idx4], 0):
                        continue
                        
                    print("m diagonal not working", m1, m2, m3, m4, mat_flat[idx1, idx2, idx3, idx4])
                    consistent = False
            else:
                if (m1 != m3 or m2 != m4) and (m1 != m4 or m2 != m3):
                    if np.isclose(mat_flat[idx1, idx2, idx3, idx4], 0):
                        continue
                        
                    print("m diagonal not working - asym")
                    consistent = False

        # print(idx1, idx2)
        # print(unf1, unf2)
        # print(mat[n1, l1, twoj1_idx, n2, l2, twoj2_idx])
        # print(mat_flat[idx1, idx2])
        val = mat[n1, l1, twoj1_idx, n2, l2, twoj2_idx, n3, l3, twoj3_idx, n4, l4, twoj4_idx]
        val_flat = mat_flat[idx1, idx2, idx3, idx4]
        if not np.isclose(val, val_flat):
            consistent = False
            #print("WTF 2")

print("4D flattening is consistent:", consistent)

Both flattening systems equivalent: True
(18, 18, 18, 18)
4D flattening is consistent: True
